In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
#reading in the data

routes = pd.read_csv('routes.txt')

trips = pd.read_csv('trips.txt')

stops = pd.read_csv('stop_times.txt', low_memory=False)

calendar = pd.read_csv('calendar.txt')
exceptions = pd.read_csv('calendar_dates.txt')


In [4]:
# reducing stops table to necessary data only

stops.columns

stops = stops[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'shape_dist_traveled','timepoint']]

# Select only the rows with the minimum and maximum 'stop_sequence' for each unique 'trip_id'
min_max_stops = stops.loc[stops.groupby('trip_id')['stop_sequence'].idxmin()]
min_max_stops = min_max_stops.append(stops.loc[stops.groupby('trip_id')['stop_sequence'].idxmax()])
min_max_stops

C:\Users\oxant\AppData\Local\Temp\ipykernel_10488\3676927779.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  min_max_stops = min_max_stops.append(stops.loc[stops.groupby('trip_id')['stop_sequence'].idxmax()])


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,timepoint
44777,1,10:10:00,10:10:00,14004939,1,0.000000,1
44458,10,17:58:00,17:58:00,14004939,1,0.000000,1
202144,100,06:45:00,06:45:00,14001490,1,0.000000,1
157743,1000,11:56:00,11:56:00,14003763,1,0.000000,1
157718,1001,10:56:00,10:56:00,14003763,1,0.000000,1
...,...,...,...,...,...,...,...
159476,995,17:46:00,17:46:00,14002530,23,10620.357162,1
159371,996,14:46:00,14:46:00,14002530,23,10620.357162,1
157842,997,15:16:00,15:16:00,14002530,25,8782.111968,1
157817,998,14:16:00,14:16:00,14002530,25,8782.111968,1


In [6]:
# Calculate the time difference between the first and last stop sequence for each trip_id and convert to hours
min_max_stops['trip_duration'] = min_max_stops.groupby('trip_id')['departure_time'].transform(lambda x: (pd.to_datetime(x).max() - pd.to_datetime(x).min()).total_seconds() / 3600)

min_max_stops

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,timepoint,trip_duration
44777,1,10:10:00,10:10:00,14004939,1,0.000000,1,0.650000
44458,10,17:58:00,17:58:00,14004939,1,0.000000,1,0.683333
202144,100,06:45:00,06:45:00,14001490,1,0.000000,1,0.650000
157743,1000,11:56:00,11:56:00,14003763,1,0.000000,1,0.333333
157718,1001,10:56:00,10:56:00,14003763,1,0.000000,1,0.333333
...,...,...,...,...,...,...,...,...
159476,995,17:46:00,17:46:00,14002530,23,10620.357162,1,0.366667
159371,996,14:46:00,14:46:00,14002530,23,10620.357162,1,0.366667
157842,997,15:16:00,15:16:00,14002530,25,8782.111968,1,0.333333
157817,998,14:16:00,14:16:00,14002530,25,8782.111968,1,0.333333


In [10]:
# calculate bus speed

min_max_stops['speed'] = min_max_stops['shape_dist_traveled']/(1000*min_max_stops['trip_duration'])

count    14134.000000
mean        13.301917
std         14.185956
min          0.000000
25%          0.000000
50%          6.993813
75%         25.352115
max         65.767724
Name: speed, dtype: float64

In [11]:
# Now select only entries for the last stop, remove entries for the 1st stop of the trip
min_max_stops = min_max_stops[min_max_stops['stop_sequence'] != 1]
min_max_stops.describe()

,stop_id,stop_sequence,shape_dist_traveled,timepoint,trip_duration,speed
count,7.067000e+03,7067.000000,7067.000000,7067.000000,7067.000000,7067.000000
mean,1.400436e+07,37.516061,19987.086145,0.998868,0.705931,26.603834
std,2.778971e+03,15.613480,18288.929848,0.033629,0.407792,6.969872
min,1.400106e+07,4.000000,3808.110805,0.000000,0.100000,13.987626
25%,1.400253e+07,28.000000,10406.747951,1.000000,0.416667,22.627042
50%,1.400253e+07,34.000000,12975.730830,1.000000,0.533333,25.352968
75%,1.400799e+07,42.000000,25882.780541,1.000000,0.833333,28.080060
max,1.400897e+07,77.000000,135294.119197,1.000000,2.566667,65.767724


In [13]:
min_max_stops.loc[min_max_stops['speed'].idxmax()]

trip_id                        6323
arrival_time               14:40:00
departure_time             14:40:00
stop_id                    14002221
stop_sequence                    13
shape_dist_traveled    43845.149098
timepoint                         1
trip_duration              0.666667
speed                     65.767724
Name: 1258, dtype: object

In [17]:
routes.describe()

,agency_id,route_id,route_desc,route_type,route_url,route_sort_order,min_headway_minutes,eligibility_restricted,continuous_pickup,continuous_drop_off,tts_route_short_name,tts_route_long_name
count,38.0,38.000000,0.0,38.000000,0.0,38.000000,3.000000,38.0,38.0,38.0,0.0,0.0
mean,558.0,8549.131579,NaN,2.973684,NaN,30.315789,70.000000,0.0,1.0,1.0,NaN,NaN
std,0.0,10158.345578,NaN,0.162221,NaN,23.013633,45.825757,0.0,0.0,0.0,NaN,NaN
min,558.0,3771.000000,NaN,2.000000,NaN,0.000000,30.000000,0.0,1.0,1.0,NaN,NaN
25%,558.0,4620.500000,NaN,3.000000,NaN,9.250000,45.000000,0.0,1.0,1.0,NaN,NaN
50%,558.0,4655.000000,NaN,3.000000,NaN,25.000000,60.000000,0.0,1.0,1.0,NaN,NaN
75%,558.0,9032.750000,NaN,3.000000,NaN,51.750000,90.000000,0.0,1.0,1.0,NaN,NaN
max,558.0,65445.000000,NaN,3.000000,NaN,74.000000,120.000000,0.0,1.0,1.0,NaN,NaN


In [33]:

print(trips['trip_id'].dtype)
print(min_max_stops['trip_id'].dtype)
min_max_stops['trip_id'] = min_max_stops['trip_id'].astype(int)

int64
object


C:\Users\oxant\AppData\Local\Temp\ipykernel_10488\1241643285.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  min_max_stops['trip_id'] = min_max_stops['trip_id'].astype(int)


In [36]:
# Merge dataframes based on 'rounte_id' column with 'outer' join
stops_trips = pd.merge(trips[['route_id', 'service_id', 'trip_id', 'trip_short_name', 'trip_headsign', 'direction_id',
                             'block_id', 'shape_id', 'operator_id']], min_max_stops, on=['trip_id'], how='outer')

stops_trips.head(20)

,route_id,service_id,trip_id,trip_short_name,trip_headsign,direction_id,block_id,shape_id,operator_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,timepoint,trip_duration,speed
0,11712,WKO00_MTWThF_20211220_20220131_1,6985,NaN,O'Sullivan Dr,1,6985,11712,38,14:50:00,14:50:00,14001676,27,19467.197481,1,0.833333,23.360637
1,11712,WKO00_MTWThF_20211220_20220131_1,6984,NaN,O'Sullivan Dr,1,6984,11712,38,09:50:00,09:50:00,14001676,27,19467.197481,1,0.833333,23.360637
2,11712,WKO00_MTWThF_20211220_20220131_1,6989,NaN,O'Sullivan Dr,1,6989,11712,38,10:50:00,10:50:00,14001676,27,19467.197481,1,0.833333,23.360637
3,11712,WKO00_MTWThF_20211220_20220131_1,6988,NaN,O'Sullivan Dr,1,6988,11712,38,11:50:00,11:50:00,14001676,27,19467.197481,1,0.833333,23.360637
4,11712,WKO00_MTWThF_20211220_20220131_1,6987,NaN,O'Sullivan Dr,1,6987,11712,38,12:50:00,12:50:00,14001676,27,19467.197481,1,0.833333,23.360637
5,11712,WKO00_MTWThF_20211220_20220131_1,6986,NaN,O'Sullivan Dr,1,6986,11712,38,13:50:00,13:50:00,14001676,27,19467.197481,1,0.833333,23.360637
6,11712,WKO00_MTWThF_20211004_20211219_1,6990,NaN,O'Sullivan Dr,1,6990,11712,38,13:50:00,13:50:00,14001676,27,19467.197481,1,0.833333,23.360637
7,11712,WKO00_MTWThF_20211004_20211219_1,6991,NaN,O'Sullivan Dr,1,6991,11712,38,09:50:00,09:50:00,14001676,27,19467.197481,1,0.833333,23.360637
8,11712,WKO00_MTWThF_20211004_20211219_1,6992,NaN,O'Sullivan Dr,1,6992,11712,38,10:50:00,10:50:00,14001676,27,19467.197481,1,0.833333,23.360637
9,11712,WKO00_MTWThF_20211004_20211219_1,6994,NaN,O'Sullivan Dr,1,6994,11712,38,12:50:00,12:50:00,14001676,27,19467.197481,1,0.833333,23.360637
